
## Background

Your familyhas asked you to use AI to help them with the following problem.



## Section 1: The digital photo generation

Your family bought their first digital camera in the 1990s-2000s and took a lot of pictures including selfies, group photos, food, scenery etc. This year, they realized you have accumulated thousands of photos and you think that it is time you sorted them out. Obviously, it is going to be tedious to be sorting so many photos and you know that you may even make some mistakes. 

So you decided to write a tool that makes use of the Image Recognition engine and  Rekognition facial APIs to help you to sort the photos into different folders.

This is an example of your folder structure.

```
OriginalPhotos\
  photo1.jpg
  photo2.jpg
  ...
  
FacesPhotos\
  Alice\
    faceofalice1.jpg
  Max\
    faceofmax1.jpg

SortedPhotos\
  ObjectsAndScenes\
    Car\
      photo1.jpg
    Beach
      photo2.jpg
  People\
    Alice\
      photo1.jpg
    Max\
      photo2.jpg
    ...
  Landmarks\
    EiffelTower\
      photo1.jpg
    Merlion\
    ...
```

The "OriginalPhotos" folder is where you will place *all* your photos taken by you in the past. You will not store them into different sub-folders.

The "FacesPhotos" folder contains a set of sub-folders, each named with the person's name. Within each sub-folder is one or more photos of the person you want to recognize.

The "SortedPhotos" folder is where you will copy photos from the "OriginalPhotos" folder into the respective photos above automatically, with help from the Objects-and-Scene Detection and Facial Recognition APIs.

In the example above, there are 2 photos in the "OriginalPhotos" folder. 
- photo1.jpg is a photo of Alice standing near a car.
- photo2.jpg is a photo of Max walking with an unknown person on the beach.

The "SortedPhotos" contains photo1.jpg copied to both the "SortedPhotos\People\Alice",  "SortedPhotos\ObjectsAndScenes\Car" and "SortedPhotos\Landmarks\EiffelTower" folders. It also contains photo2.jpg copied to both the "SortedPhotos\People\Alice" and "SortedPhotos\ObjectsAndScenes\Car" folders

### Step 1: Import Libraries

Import the neccessary libaries.

Use the key provided in class and remember to use your admin number as your collection id.


In [1]:
import boto3
import os
from botocore.exceptions import ClientError
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from PIL import Image, ImageDraw
from io import BytesIO
import io
from PIL import Image

collectionId = '19A'
region = 'ap-southeast-2'
access_key = 'A'
secret_key = 'Z'


In [10]:
googleAPIKey = "A"

import requests 
import base64
from botocore.exceptions import ClientError
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from PIL import Image, ImageDraw
from io import BytesIO

googleurl = "https://vision.googleapis.com/v1/images:annotate?key=" + googleAPIKey
googleheaders = {'Content-Type': 'application/json'}

### Step 2: Create the client object to send request to the Amazon Web Services. 


In [11]:
client = boto3.client('rekognition',
    region_name = region,
    aws_access_key_id = access_key,
    aws_secret_access_key = secret_key)


### Step 3: Create a new collection of faces of people who you want to recognise


In [40]:
# Delete the collection (if it already exists)
try:
    client.delete_collection(CollectionId = collectionId)
    print ('Collection deleted.')
except ClientError as e:
    print ('Collection deleted.')
    
# then we create a new collection with that same name.
try:
    response = client.create_collection(CollectionId = collectionId)
    print ('Collection created.')
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
        print ('The collection ' + collectionId + ' already exists. Run the Python code in the Delete Collection section below.')
    else:
        print ('Some other error has occured.')


Collection deleted.
Collection created.


### Step 4: Index Photos of 3-4 People Using Amazon Rekognition 

Using the small set of photos (3-4 photos per person) that you have gathered, place the pictures in individual folders. Write codes to upload the photos to Amazon to index those faces. 

*NOTE: You may use your personal photos, but you can also use photos of other people obtained from the internet. Each learner should have a unique set of photos from others.*


In [41]:
# Index photographs of faces found under the FacesPhotos folder #
files = []

# loop through the files and folders under P06-data
#
for r, d, f in os.walk('FacesPhotos'):
    
    # Each folder created under data contains photos of 
    # the same person. And we will treat the folder name as
    # an identifier for that person.
    #
    for personId in d:
        
        # Loop through the files in the data folder
        #
        for r1, d1, f1 in os.walk('FacesPhotos\\' + personId):
            
            # Retrieve the file name of each file
            #
            for file in f1:
                
                imageFilename = 'FacesPhotos\\' + personId + '\\' + file
                print ('Indexing: ' + imageFilename)
                
                # Request Amazon to index the face found in the file.
                #
                with open(imageFilename, 'rb') as image:
                    response = client.index_faces(
                        CollectionId = collectionId, 
                        Image = {'Bytes': image.read()}, 
                        ExternalImageId = personId,
                        MaxFaces = 1,
                        QualityFilter = "AUTO",
                        DetectionAttributes = ["ALL"])  

print ('Indexing complete.')


Indexing: FacesPhotos\Kate\faceofKate1.jpg
Indexing: FacesPhotos\Kate\faceofKate2.jpg
Indexing: FacesPhotos\MdmLee\faceofMdmLee1.jpg
Indexing: FacesPhotos\MdmLee\faceofMdmLee2.jpg
Indexing: FacesPhotos\Modi\faceofModi1.jpg
Indexing: FacesPhotos\Modi\faceofModi2.jpeg
Indexing: FacesPhotos\Son\faceofSon1.jpg
Indexing: FacesPhotos\Son\faceofSon2.jpg
Indexing complete.


### Step 5: Copy Photos to Folder Based on Attributes and Faces Recognized

Collect a set of about 20 photos and place them into a folder and send each photo to either:
- Amazon Rekognition Object-and-Scene Detection OR 
- Google's Cloud Vision API for Object-and-Scene and Landmark Detection
for recognition

If the results shows that there is a Human/Person/Man:
- Send the pictures to Amazon Facial Recognition.

Using the attributes returned from Object-and-Scene Detection API (only those whose confidence > 95%), duplicate the picture to the relevant folders.

Using recognized face match result returned from the Facial Recognition API (face match threshold > 90%), duplicate the picture to the relevant folders.

The cell below shows an example skeleton of how your tool should be coded.

In [60]:
# Codes below shows how to copy a file from one folder to another 
#
import os
import shutil

# This function copies a file provided in the original file path
# to the destination folder.
#
# If the destination folder does not exist, it will create it.
#
def copy(original_file_path, destination_folder_path):
    if not os.path.exists(destination_folder_path):
        os.makedirs(destination_folder_path)
    shutil.copy(original_file_path,  destination_folder_path)


In [ ]:
# Loop through the files in your OriginalPhotos folder using google
for r, d, files in os.walk('OriginalPhotos'):

    for filename in files:
        
        # Load the image bytes that we will use to send to Amazon later.
        photo = 'OriginalPhotos\\' + filename 
       
        print("\nPicture To Process: %s" % photo)
        # Load the actual image that can be used to crop out the faces later
        image = Image.open(photo)
        stream = io.BytesIO()
        image.save(stream,format="JPEG")
        image_binary = stream.getvalue()

        # Send the image bytes to the Object-and-Scene (and Landmark Detection for additional marks) API
        # https://docs.aws.amazon.com/rekognition/latest/dg/images-bytes.html#w696aac18b7c19c16b4b2b3
        
        # ToDo: Construct the data to Google
        #
        data = {
            'requests': 
            [
                {
                    'image': { 'content': base64.b64encode(image_binary).decode("utf-8") },
                    'features': [ { 'type': 'OBJECT_LOCALIZATION' }, 
                                 { 'type': 'LABEL_DETECTION' },
                                 { 'type': 'LANDMARK_DETECTION' }]
                }
            ]
        }

        # ToDo: Send the image data to Google to recognize
        #
        r = requests.post(url = googleurl, headers = googleheaders, json = data) 

        # Check and display the results
        if r.status_code == 200:
            result = r.json()

            #print (result)

            # ToDo: Write a for-loop to loop through all the 'responses'.
            #
            for response in result['responses']:

                # Do a sanity check, in case the key doesn't exist in the dictionary
                #
                if 'landmarkAnnotations' in response:
                    for annotation in response['landmarkAnnotations']:
                        if float(annotation['score']) * 100 > 80.0:
                            print("Landmarks %4.2f %s" % (annotation['score'] * 100, annotation['description']))
                            landMarks(photo,annotation["description"])
                            copy("OriginalPhotos\\" + filename, "SortedPhotos\\LandMarks\\" + annotation['description'])
                            

                # Do a sanity check, in case the key doesn't exist in the dictionary
                #
                if 'localizedObjectAnnotations' in response:
                    for annotation in response['localizedObjectAnnotations']:
                        
                        if float(annotation['score']) * 100 > 80.0:
                            print("ObjectsAndScenes %4.2f %s" % (annotation['score'] * 100, annotation['name']))
                            
                            if(annotation['name']=='Person' or annotation['name']=='Woman' or annotation['name']=='Man'):

                                # Detect faces in the photo
                                response1 = client.detect_faces(
                                Image={'Bytes':image_binary}                                        
                                    )

                                all_faces=response1['FaceDetails']
                                # Initialize list object
                                boxes = []

                                # Get image diameters
                                image_width = image.size[0]
                                image_height = image.size[1]

                                # Loop through all detected faces in the photo
                                for face in all_faces:
                                    # Crop out the faces from the photo
                                    box=face['BoundingBox']
                                    x1 = int(box['Left'] * image_width) * 0.9
                                    y1 = int(box['Top'] * image_height) * 0.9
                                    x2 = int(box['Left'] * image_width + box['Width'] * image_width) * 1.10
                                    y2 = int(box['Top'] * image_height + box['Height']  * image_height) * 1.10
                                    image_crop = image.crop((x1,y1,x2,y2))

                                    stream = io.BytesIO()
                                    image_crop.save(stream,format="JPEG")
                                    image_crop_binary = stream.getvalue()

                                    # Send the cropped photo to Amazon Facial Recognition API
                                    response2 = client.search_faces_by_image(
                                            CollectionId=collectionId,
                                            Image={'Bytes':image_crop_binary},
                                            MaxFaces = 5,
                                            FaceMatchThreshold = 90
                                            )

                                    # Identify the known faces from the API and copy the photo to the appropriate folders
                                    # under your SortedPhotos folder
                                    #
                                    if(len(response2['FaceMatches']) > 0):
                                        faceMatch = response2["FaceMatches"][0]
                                        person_name=faceMatch['Face']['ExternalImageId']
                                        print("Face Recognized ", person_name)
                                        copy("OriginalPhotos\\" + filename, "SortedPhotos\\People\\" + person_name)                                            

                            else:
                                copy("OriginalPhotos\\" + filename, "SortedPhotos\\ObjectsAndScenes\\" + annotation['name'])
                            


Picture To Process: OriginalPhotos\photo01.jpeg
ObjectsAndScenes 92.09 Man
Face Recognized  Modi
Face Recognized  Modi
ObjectsAndScenes 89.30 Man
Face Recognized  Modi
Face Recognized  Modi
ObjectsAndScenes 83.89 Coat
ObjectsAndScenes 80.54 Pants

Picture To Process: OriginalPhotos\photo02.jpg
Landmarks 88.81 Liberty Island
Landmarks 88.29 Margaret Pace Park
Landmarks 80.83 Statue of Liberty National Monument
ObjectsAndScenes 93.75 Glasses
ObjectsAndScenes 85.57 Person
Face Recognized  Son
Face Recognized  MdmLee
ObjectsAndScenes 82.30 Person
Face Recognized  Son
Face Recognized  MdmLee

Picture To Process: OriginalPhotos\photo03.jpg
Landmarks 86.32 Niterói Contemporary Art Museum
Landmarks 82.64 Niterói Contemporary Art Museum
ObjectsAndScenes 84.68 Person
Face Recognized  MdmLee
ObjectsAndScenes 83.24 Woman
Face Recognized  MdmLee

Picture To Process: OriginalPhotos\photo04.jpg
Landmarks 81.47 Dancing House
ObjectsAndScenes 92.41 Glasses
ObjectsAndScenes 81.57 Person
Face Recognized

## Section 2: Submission

### Submission Requirements

Please submit the following:

1. Entire folder structure containing photos of 3-4 people used to index faces
2. Entire folder structure containing 20 photos of the 3-4 people in different scenes and backgrounds
3. A copy of this Jupyter Notebook containing your work.

---

### Codes on how to copy files from one directory to another

In [17]:
# Codes below shows how to copy a file from one folder to another 
#
import os
import shutil

# This function copies a file provided in the original file path
# to the destination folder.
#
# If the destination folder does not exist, it will create it.
#
def copy(original_file_path, destination_folder_path):
    if not os.path.exists(destination_folder_path):
        os.makedirs(destination_folder_path)
    shutil.copy(original_file,  destination_folder_path)

'''
# The following is an example of how to use the copy function
#
filename = 'rose.jpg'

# Note that in Python strings, the backslash "\" is a special character, also called the "escape" character. 
# It is used in representing certain whitespace characters: 
# "\t" is a tab, "\n" is a newline, and "\r" is a carriage return 
#

original_folder =  PureWindowsPath("C:\\TMP\\old_folder") 
new_folder = PureWindowsPath("C:\TMP\\new_folder")
original_file = original_folder.joinpath(filename)

copy(original_file, new_folder)
'''



'\n# The following is an example of how to use the copy function\n#\nfilename = \'rose.jpg\'\n\n# Note that in Python strings, the backslash "" is a special character, also called the "escape" character. \n# It is used in representing certain whitespace characters: \n# "\t" is a tab, "\n" is a newline, and "\r" is a carriage return \n#\n\noriginal_folder =  PureWindowsPath("C:\\TMP\\old_folder") \nnew_folder = PureWindowsPath("C:\\TMP\\new_folder")\noriginal_file = original_folder.joinpath(filename)\n\ncopy(original_file, new_folder)\n'